# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import zipfile
import pandas as pd

from azureml.core import Workspace, Dataset, Experiment

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

quick-starts-ws-152247
aml-quickstarts-152247
southcentralus
81cefad3-d2c9-4f77-a466-99a7f541c7bb


In [3]:
# Create compute cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
dataset = Dataset.get_by_name(ws, name='Ames-housing-dataset')
print(dataset.to_pandas_dataframe().info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 no

In [5]:
# Choose a name for experiment
experiment_name = 'Ames-housing-AutoML'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
automl_settings = {
    'experiment_timeout_minutes' : 120,
    'n_cross_validations' : 3,
    'enable_early_stopping' : True,
    'iteration_timeout_minutes' : 5,
    'max_concurrent_iterations' : 4,
    'max_cores_per_iteration' : -1
}

automl_config = AutoMLConfig(
    task='regression',
    primary_metric='normalized_root_mean_squared_error',
    compute_target=cpu_cluster,
    training_data=dataset,
    label_column_name='SalePrice',
    **automl_settings)

In [7]:
from azureml.widgets import RunDetails
# TODO: Submit your experiment
aml_run = experiment.submit(automl_config)
RunDetails(aml_run).show()
aml_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Ames-housing-AutoML,AutoML_cd63750f-df45-4803-b135-af0f23de09cd,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
Ames-housing-AutoML,AutoML_cd63750f-df45-4803-b135-af0f23de09cd,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      
+---------------------------------+---------------------------------+---------------------------------+
|Column name              

{'runId': 'AutoML_cd63750f-df45-4803-b135-af0f23de09cd',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-23T08:25:19.30697Z',
 'endTimeUtc': '2021-07-23T08:47:05.349103Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Ames-housing-AutoML","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-152247","workspace_name":"quick-starts-ws-152247","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
# Get your best run and save the model from that run.
best_run, fitted_model = aml_run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-core, training version:1.32.0, current version:1.31.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-defaults, training version:1.32.0, current version:1.31.0
Package:azureml-interpret, training version:1.32.0, current version:1.31.0
Package:azureml-mlflow, training version:1.32.0, current version:1.31.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.31.0
Package:azureml-telemetry, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-runtime, training version:1.32.0, current version:1.31.0


Run(Experiment: Ames-housing-AutoML,
Id: AutoML_cd63750f-df45-4803-b135-af0f23de09cd_36,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
)), ('elasticnet', ElasticNet(alpha=0.05357894736842105, copy_X=True, fit_intercept=True, l1_ratio=0.6352631578947369, max_iter=1000, normalize=False, positive=False, precompute=False, random_state=None, selection='cyclic', tol=0.0001, warm_start=False))], verbose=False))], weights=[0.4, 0.13333333333333333, 0.26666666666666666, 0.06666666666666667, 0.13333333333333333]))],
                                     verbo

In [9]:
best_run_metrics = best_run.get_metrics()
best_run_metrics

{'r2_score': 0.885898472424608,
 'normalized_root_mean_squared_log_error': 0.042947817229549566,
 'mean_absolute_error': 15439.116472115917,
 'normalized_root_mean_squared_error': 0.037139291460024836,
 'spearman_correlation': 0.9573852130501542,
 'median_absolute_error': 9654.516791774067,
 'root_mean_squared_error': 26744.00378036388,
 'mean_absolute_percentage_error': 9.122770171936464,
 'normalized_mean_absolute_error': 0.021440239511339974,
 'explained_variance': 0.886052769069071,
 'root_mean_squared_log_error': 0.13203033398097183,
 'normalized_median_absolute_error': 0.013407188990104244,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_cd63750f-df45-4803-b135-af0f23de09cd_36/residuals',
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_cd63750f-df45-4803-b135-af0f23de09cd_36/predicted_true'}

In [10]:
import json
with open('best_run_metrics.json', 'w') as file:
    json.dump(best_run_metrics, file)
with open('best_run_details.txt', 'w') as file:
    file.write(str(best_run.get_details()))

In [13]:
# Test the best model accuracy in the Kaggle competition
test = pd.read_csv('test.csv', index_col='Id')
test.shape

(1459, 79)

In [26]:
# NaN values not treated by the model .... 
y_hat = fitted_model.predict(test.fillna(0))
y_hat[:10]

array([124776.48048299, 160882.70997397, 186544.04940138, 189789.33383424,
       188952.25610385, 173681.83574201, 172458.43604907, 163224.08993769,
       187574.6388713 , 132248.48694095])

In [27]:
len(y_hat)

1459

In [33]:
with open('automl-submission.csv', 'w') as file:
    pd.DataFrame({'Id' : test.index.to_list(), 'SalePrice': y_hat}).to_csv(file, index=False)

In [34]:
os.listdir()

['.amlignore',
 '.amlignore.amltmp',
 '.azureml',
 '.config',
 '.ipynb_checkpoints',
 'automl-submission.csv',
 'automl.ipynb',
 'automl.log',
 'azureml_automl.log',
 'best_run_details.txt',
 'best_run_metrics.json',
 'data_description.txt',
 'house-prices-advanced-regression-techniques.zip',
 'Register_dataset.ipynb',
 'sample_submission.csv',
 'test.csv',
 'train.csv']

In [36]:
# Set the enviroment variables
os.environ['KAGGLE_USERNAME'] = "lubomrstraka"
os.environ['KAGGLE_KEY'] = "c7347462ef834e6645ce238c2f2fa561"
! kaggle competitions submit -c house-prices-advanced-regression-techniques -f automl-submission.csv -m "Submission generated by Azure Automated ML."

100%|██████████████████████████████████████| 33.7k/33.7k [00:01<00:00, 27.0kB/s]
Successfully submitted to House Prices - Advanced Regression Techniques

> 2021-07-23: Your submission scored 0.13612, which is not an improvement of your best score (0.12481).

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [37]:
# Check the path to the model
best_run.get_file_names()[-6] # where is output/model.joblib ?[-4]? CHECK it in the next cell

'outputs/model.pkl'

In [45]:
# Save the best model
from azureml.core import Model
os.makedirs('./outputs/', exist_ok=True)
best_run.download_file(best_run.get_file_names()[-6], output_file_path='./outputs/')
model = Model.register(ws, model_path='outputs/model.pkl', model_name='Ames-Housing-AutoML-Model', tags=best_run_metrics)
print(model.name, model.id, model.version, sep='\t')

Registering model Ames-Housing-AutoML-Model
Ames-Housing-AutoML-Model	Ames-Housing-AutoML-Model:1	1


In [50]:
%%writefile score.py
import json
import numpy an np
import os
import pickle
import joblib

def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')

Overwriting score.py


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [46]:
# Delete() is used to deprovision and delete the AmlCompute target. 
cpu_cluster.delete()